## Merge all generated data from doubao model

In [3]:
from modelzipper.tutils import *
import re
from pprint import pprint
import itertools
import random
import transformers
from datasets import Dataset, concatenate_datasets, DatasetDict, load_from_disk, load_dataset
import pandas as pd
import numpy as np

In [2]:
dataset = auto_read_dir("/data/zecheng/data/process_wiki_document/one_hop", file_suffix=".jsonl", file_prefix="generated_QA_pairs_thread")
all_data = [auto_read_data(f) for f in dataset]
all_data = list(itertools.chain(*all_data))
meta_data = auto_read_data("/data/zecheng/data/process_wiki_document/one_hop/statistic.json")

# convert to group_map
group_map = {}
for k, v in meta_data.items():
    for g in meta_data[k]:
        group_map[g] = int(k)

number of files with prefix 'generated_QA_pairs_thread' and suffix '.jsonl': 16


In [5]:
CHUNK_NUM = 4

def extract_qa_pairs(text):
    sp_text = text.split("####")
    if len(sp_text) < 5:
        return None
    question, answer, reference = text.split("####")[1].split('Q:')[-1].strip(), text.split("####")[2].split('A:')[-1].strip(), text.split("####")[3].split('R:')[-1].strip()
    return question, answer, reference

def combine_data(data, chunk_num):
    combined_data = []
    for i in range(0, len(data), chunk_num):
        combined_data.append(data[i:i+chunk_num])
    return combined_data

processed_data = []
for item in all_data:
    context, qa_pairs, id = item['id'], item['QA_pairs'], item["reference"]
    res = extract_qa_pairs(qa_pairs)
    if res is None:
        continue
    question, answer, ref = res[0], res[1], res[2]
    processed_data.append({"context": context, "question": question, "answer": answer, "ref": ref, "id": int(id), "wiki_source_id": group_map[int(id) + 1]})

auto_save_data(processed_data, "/data/zecheng/data/process_wiki_document/one_hop/merged_data/processed_data_2.jsonl")
combined_data = combine_data(processed_data, chunk_num=CHUNK_NUM)
# print(f" num of combined data: {len(combined_data)} \n num of processed data: {len(processed_data)}")
# auto_save_data(combined_data, "/data/zecheng/data/process_wiki_document/one_hop/combined_data.jsonl")

jsonl file saved successfully!


In [9]:
def random_select_from_combined_data(all_samples, num_cases=8, selected_cases=1):

    case_ids = [int(item["id"]) for item in all_samples]
    combinations_list = list(itertools.combinations(case_ids, selected_cases))
    batch_data = []
    # ref_lst = [item['context'] for item in all_samples]
    # all_wiki_src_id = [item["wiki_source_id"] for item in all_samples]
    for item in combinations_list:
        chosen_id = item[0]  
        remain_case_ids = list(set(case_ids) - set((chosen_id,)))
        k = min(len(remain_case_ids)-1, 4)
        reject_id = random.sample(remain_case_ids, k)
        for rej_id in reject_id:
            cur_sample = {
                "all_ref_ids": case_ids,
                "chosen_id": chosen_id,
                "rejected_id": rej_id,
            }
            batch_data.append(cur_sample)
    return batch_data

all_created_cases = []
with tqdm(total=len(combined_data)) as pbar:
    for c_data in combined_data:
        batch_data = random_select_from_combined_data(c_data, num_cases=len(c_data), selected_cases=1)
        all_created_cases += batch_data
        pbar.update(1)
# pprint(all_created_cases[0])
print(f"finish, current data sample nums: {len(all_created_cases)}")

100%|██████████| 2400/2400 [00:00<00:00, 30853.23it/s]

finish, current data sample nums: 19192


### combine and save all data into datasets format

In [4]:
from datasets import Dataset

data_dict = {key: [dic[key] for dic in all_created_cases] for key in all_created_cases[0]}
dataset = Dataset.from_dict(data_dict)
dataset.save_to_disk("/data/zecheng/data/process_wiki_document/one_hop/merged_data/combine_data_hf_2", num_proc=16)

NameError: name 'all_created_cases' is not defined

In [5]:
import spacy
nlp = spacy.load('en_core_web_sm')

text = "first 80 sec."
doc = nlp(text)

meaningful_labels = {"PERSON", "ORG", "GPE", "MONEY", "PRODUCT"}
for ent in doc.ents:
    if ent.label_ in meaningful_labels:
        print(ent.text, ent.label_)

ModuleNotFoundError: No module named 'spacy'

In [5]:
data = auto_read_data("/data/zecheng/data/process_wiki_document/one_hop/merged_data/processed_data_2.jsonl")
print(len(data))
data[0]

9598


{'context': '<s> = Fatboy (EastEnders) =\nArthur" Fatboy" Chubb (also credited as Fat Boy) is a fictional character from the BBC soap opera EastEnders and its Internet spin off EastEnders: E20, played by Ricky Norwood. He made his first appearance in EastEnders on 5 January 2010 before appearing in the spin off. Fatboy is one of four main characters in the first series of E20 and makes cameo appearances in the second and third series. He was created by the EastEnders: E20 writing team during a BBC summer school in August 2009. He frequently uses London street slang and is described as brash, confident, caring, a hustler and a womaniser. Critics have called him zany, an asset to the show, authentic, a cliche, and yet to serve a purpose, with Stuart Heritage from The Guardian calling for him to be axed. Norwood won the Best Newcomer award at the 2010 Inside Soap Awards and Most Popular Newcomer at the 2011 National Television Awards for his portrayal of Fatboy, and has been nominated for